In [1]:
from itertools import combinations

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from itertools import combinations

import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import re
import warnings

import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from unidecode import unidecode
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from feature_engine import categorical_encoders as ce

from itertools import combinations
from datetime import datetime
from contextlib import contextmanager

In [2]:
NUM_BOOST_ROUND= 10000

DATE = ["Field_{}".format(i) for i in [5, 6, 7, 8, 9, 11, 15, 25, 32, 33, 34, 35, 40]]
DATETIME = ["Field_{}".format(i) for i in [1, 2, 43, 44]]

In [3]:
def subtract_date(date1,date2, df):
    df[date1] = pd.to_datetime(df[date1], infer_datetime_format=True)
    df[date2] = pd.to_datetime(df[date2], infer_datetime_format=True)
    df[date1+date2] = (df[date2] - df[date1]).dt.days
def process_ngaySinh(s):
    if s != s:
        return np.nan
    try:
        s = int(s)
    except ValueError:
        s = s.split(" ")[0]
        
    return datetime.strptime(str(s)[:6], "%Y%m")
def datetime_normalize(s):
    if s != s:
        return np.nan
    
    s = s.split(".")[0]
    if s[-1] == "Z":
        s = s[:-1]
        
    date, time = s.split("T")
    datetime_obj = datetime.strptime(s, "%Y-%m-%dT%H:%M:%S")
    return datetime_obj
def date_normalize(s):
    if s != s:
        return np.nan
    
    try:
        datetime_obj = datetime.strptime(s, "%m/%d/%Y")
    except:
        datetime_obj = datetime.strptime(s, "%Y-%m-%d")
        
    return datetime_obj
def process_datetime_cols(df):
    cat_cols = []
    for col in DATETIME:
        df[col] = df[col].apply(datetime_normalize)
        
    for col in DATE:
        if col == "Field_34":
            continue
        df[col] = df[col].apply(date_normalize)

    df["Field_34"] = df["Field_34"].apply(process_ngaySinh)
    
    cat_cols += DATE + DATETIME
    for col in DATE + DATETIME:
        df[col] = df[col].dt.strftime('%d-%m-%Y')
    
    subtract_date('Field_5','Field_6',df)
    subtrac_List = ['Field_1', 'Field_2', 'Field_43', 'Field_44', 'Field_7','Field_8', 'Field_9']
    subtract_2C = list(combinations(subtrac_List, 2))
    for l in subtract_2C:
        subtract_date(l[0],l[1],df)
    
    for cat in ['F', 'E', 'C', 'G', 'A']:
        subtract_date(f'{cat}_startDate', f'{cat}_endDate', df)
    print(df.shape) 
    return df
def str_normalize(s):
    s = str(s).strip().lower()
    s = re.sub(' +', " ", s)
    return s
def combine_gender(s):
    x, y = s
    return x if x != None else y if y != None else np.nan
def process_gender(df):
    df["gender"] = df[["gioiTinh", "info_social_sex"]].apply(combine_gender, axis=1)
    df["country"] = df[['currentLocationCountry', 'homeTownCountry']].apply(combine_gender, axis=1)
    df["city"] = df[['currentLocationState', 'homeTownState']].apply(combine_gender, axis=1)
    return df
def process_ordinal(df):        
    df["subscriberCount"].replace(0, np.nan, inplace=True)
    df["friendCount"].replace(0, np.nan, inplace=True)
    
    df["Field_13"] = df["Field_13"].apply(lambda x: 1 if x == x else 0)
    df["Field_38"] = df["Field_38"].map({0: 0.0, 1: 1.0, "DN": np.nan, "TN": np.nan, "GD": np.nan})
    df["Field_62"] = df["Field_62"].map({"I": 1, "II": 2, "III": 3, "IV": 4, "V": 5, "Ngoài quốc doanh Quận 7": np.nan})
    df["Field_47"] = df["Field_47"].map({"Zezo": 0, "One": 1, "Two": 2, "Three": 3, "Four": 4})
    
    df["Field_27"] = df["Field_27"].replace({0.0: np.nan})
    df["Field_28"] = df["Field_28"].replace({0.0: np.nan})
        
    for col in df.columns:
        if df[col].dtype.name == "object":
            df[col] = df[col].apply(str_normalize).astype("category")
            
    return df
def transform(df):
    df = process_datetime_cols(df)
    df = process_gender(df)
    df = process_ordinal(df)
    return df

In [ ]:
df_train = pd.read_csv("../../data/kalapa/train_new.csv")
df_test = pd.read_csv("../../data/kalapa/test_new.csv")
df_all = df_train.drop(['label'], 1).append(df_test)


df_all_fe = transform(df_all.copy())

df_all_fe['Age'] = df_all_fe["ngaySinh 1"].apply(lambda x: 2020 - x)
df_all_fe = df_all_fe.drop('ngaySinh 1', axis = 1)
cols_select = [x for x in df_all_fe.columns if x not in DATE + DATETIME  + [f'{cat}_endDate' for cat in ['F', 'E', 'C', 'G', 'A']] + [f'{cat}_startDate' for cat in ['F', 'E', 'C', 'G', 'A']]]
df_fe = df_all_fe[cols_select]
df_fe.replace([np.inf, -np.inf], -99999, inplace=True)
df_fe["sum_null"] = df_fe.isnull().sum(axis=1)
df_fe.drop(['info_social_sex', 'gioiTinh'], axis = 1)
df_fe.Thue  = df_fe.Thue.apply(lambda x: "no" if x == "nan" else "yes")
y_label = df_train["label"]
df_fe["Diachi"] = df_fe.Field_48.astype("object")
for i in range(len(df_fe)):
    if df_fe.Field_48.iloc[i] == "nan" and df_fe.Field_49.iloc[i] == "nan":
        df_fe["Diachi"].iloc[i] = np.nan
    elif df_fe.Field_48.iloc[i] == "nan" and df_fe.Field_49.iloc[i] != "nan":
        df_fe["Diachi"].iloc[i] = "True"
    elif df_fe.Field_48.iloc[i] != "nan" and df_fe.Field_49.iloc[i] == "nan":
        df_fe["Diachi"].iloc[i] = "True"
    else:
        if df_fe.Field_48.iloc[i] == df_fe.Field_49.iloc[i]:
            df_fe["Diachi"].iloc[i] = "True"
        else:
            df_fe["Diachi"].iloc[i] = "False"
df_fe = df_fe.drop(["Field_48", "Field_49", "currentLocationCountry", "homeTownCountry", \
                    "currentLocationState", "friendCount", "subscriberCount", \
                   'homeTownState' ], axis = 1)   
train_fe = df_fe[df_fe["id"] < df_train.shape[0]]
test_fe = df_fe[df_fe["id"] >= df_train.shape[0]]

# Label-Encoding
lbl = LabelEncoder()
for col in df_fe.columns:
    if df_fe[col].dtype.name == "category" or  df_fe[col].dtype.name == "object":
        train_fe[col] = train_fe[col].astype(str)
        test_fe[col] = test_fe[col].astype(str)
        encoder = ce.CountFrequencyCategoricalEncoder(encoding_method='frequency')
        train_fe[col] = train_fe[col].fillna('None')
        test_fe[col] = test_fe[col].fillna('None')

        # Only take the common values in Train/Test set
        common_vals = list(set(train_fe[col]).intersection(set(test_fe[col])))

        # Take if vals appeared both 5 times
        #common_vals = set(train_fe[col].value_counts()[train_fe[col].value_counts() > 10].index).intersection(test_fe[col].value_counts()[test_fe[col].value_counts()>4].index)

        # Replace not-common values with "Missing" or np.nan
        train_fe.loc[~train_fe[col].isin(common_vals), col] = 'Missing'
        test_fe.loc[~test_fe[col].isin(common_vals), col] = 'Missing'

        # Implement LE
        lbl.fit(train_fe[col].tolist() + test_fe[col].tolist())
        train_fe[col] = lbl.transform(train_fe[col])
        test_fe[col] = lbl.transform(test_fe[col])

print(train_fe.shape)
print(test_fe.shape)

(73411, 203)


In [ ]:
df_train = pd.concat([df_train.id,df_train.label, train_fe.iloc[:,1:]], axis = 1)

In [ ]:
df_train

In [ ]:
data_version = "01-02-2021-v3"

In [ ]:
os.makedirs(f"../../data/kalapa/{data_version}/", exist_ok=True)
df_train.to_csv(f"../../data/kalapa/{data_version}/train.csv", index = False)
test_fe.to_csv(f"../../data/kalapa/{data_version}/test.csv", index = False)